In [1]:
!pip install transformers==4.49.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 70.5 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0


In [2]:
!python --version

Python 3.10.12


In [3]:
!pip install sentencepiece

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoProcessor
import torch
import pandas as pd

In [5]:
model_id = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
processor = AutoProcessor.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
!pip install datasets

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nvnikhil0001/data-science-questions-and-answers")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/data-science-questions-and-answers


In [5]:
import pandas as pd

data1 = pd.read_csv(path/DataScienceQA.csv)
pd.head()

NameError: name 'DataScienceQA' is not defined

In [7]:
from datasets import load_dataset

dataset = load_dataset("Harikrishnan46624/AI_QA_Data")
test = load_dataset("team-bay/data-science-qa")
dataset

README.md:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


new_train.csv:   0%|          | 0.00/473k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1125 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

DataScienceDataset.csv:   0%|          | 0.00/146k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/473 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'question', 'answer'],
        num_rows: 1125
    })
})

In [8]:
test

DatasetDict({
    train: Dataset({
        features: ['type', 'question', 'answer'],
        num_rows: 473
    })
})

In [9]:
assistant = "You are a datascience tutor. Users will ask you you data science questions and you will provide them with the answers."

import ast  # To safely evaluate the string list representation

def format_data(examples):
    inputs = [assistant + " " + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target = examples["answer"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [10]:
tokenized_dataset = dataset.map(format_data, batched=True)

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

In [11]:
data_test = test.map(format_data, batched=True)

Map:   0%|          | 0/473 [00:00<?, ? examples/s]

In [12]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [13]:
!pip install nltk rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=82dbc98ffe8adb5186197dfa73cf69bd9e152911eb11c685fb0eb30be847a26b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [14]:
# Evaluation
import evaluate
import nltk
import numpy as np



nltk.download('punkt')
metric = evaluate.load("rouge")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

In [16]:
# parameters
learning_rate = 3e-4
batch_size = 4
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
weight_decay = 0.01
num_epochs = 10

In [17]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    num_train_epochs=num_epochs,
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none"
)

In [18]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=data_test,
   processing_class = processor,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [19]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 126.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 109.12 MiB is free. Process 6080 has 15.78 GiB memory in use. Of the allocated memory 15.17 GiB is allocated by PyTorch, and 317.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.save_model("./chat")

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("die9origephit/data-science-interview-questions")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/data-science-interview-questions


In [ ]:
last_checkpoint = "/kaggle/working/results/checkpoint-1000"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

In [ ]:
my_question = "What is reinforcement learning?"
inputs = "Please answer this question: " + my_question
print(inputs)

In [ ]:
inputs = tokenizer(inputs, return_tensors="pt")
outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])
print(answer)